In [6]:
!pip3 install nltk
!python3 -m nltk.downloader wordnet punkt

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import KFold, train_test_split
from util import load_json

seed = 17

/usr/local/Cellar/python/3.7.0/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/williamzou/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/williamzou/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
articles = [article for article in load_json('processed_data_aminer.json') if len(article['references']) > 15][:4000]
articles_df = pd.DataFrame.from_dict(articles)
articles_df['title'] = articles_df['title'].str.lower().fillna('')
articles_train_df, articles_test_df = train_test_split(articles_df, test_size=0.001, random_state=17)

In [11]:
count_vectorizer = CountVectorizer()
corpus = [article['abstract'] for _, article in articles_train_df.iterrows()]
count_vectorizer.fit(corpus)

step_size = 1000
count_cols = ['vocab_{}'.format(word) for word in count_vectorizer.get_feature_names()]
counts_df = pd.DataFrame(columns=count_cols)
for i in range(0, len(articles_train_df), step_size):
  corpus = [article['abstract'] for _, article in articles_train_df.iloc[[k for k in range(i, min(i+step_size+1, len(articles_train_df)))]].iterrows()]
  counts = count_vectorizer.trnansform(corpus)
  step_counts_df = pd.DataFrame(counts.todense(), columns=count_cols)
  counts_df = pd.concat([counts_df, step_counts_df], ignore_index=True)

In [15]:
len(counts_df.columns)

18878

In [16]:
train_counts_df = pd.concat([articles_train_df, counts_df], axis=1)
train_counts_no_nans_df = train_counts_df[count_cols].fillna(0)

In [17]:
def get_test_score(model, train_df, test_df, scores, top_k=200):
  # check if top_k vectors with highest cosine similarity have one of the cited papers
  score = 0
  num_examples = len(test_df)
  score_indices = np.argsort(scores, axis=1)
  for i, (_, row) in enumerate(test_df.iterrows()):
    citations = row['references']
    if type(citations) == list:
      top_k_docs = score_indices[i][::-1][:top_k]
      top_k_docs_ids = train_df.iloc[top_k_docs]['id'].tolist()
      if any(citation in top_k_docs_ids for citation in citations):
        score += 1
      elif not any(train_df['id'].isin(citations)):
        num_examples -= 1
    else:
      num_examples -= 1
    
    print(i, num_examples, score, score / num_examples)

  return 0

In [18]:
n_splits = 2
kf = KFold(n_splits=n_splits, random_state=seed)

# grid search for best n_components
scores = []
base_n_components = 8
for n in range(base_n_components, 10):
  fold_scores = []
  for i, (train_index, test_index) in enumerate(kf.split(train_counts_no_nans_df)):
    train_df = train_counts_no_nans_df.iloc[train_index]
    validate_df = train_counts_no_nans_df.iloc[test_index]
    lda = LatentDirichletAllocation(n_components=n, random_state=seed)
    lda.fit(train_df[count_cols])
    X = lda.transform(train_counts_no_nans_df[count_cols])
    X_validate = lda.transform(validate_df[count_cols])
    score = get_test_score(lda, train_counts_df, train_counts_df.iloc[test_index], np.matmul(X_validate, X.T))
    fold_scores += [score]
    print(score)
   
  scores += [sum(fold_scores) / len(fold_scores)]

best_n_components = base_n_components + scores.index(max(scores))

0 1999 0 0.0
1 1998 0 0.0
2 1997 0 0.0
3 1996 0 0.0
4 1995 0 0.0
5 1994 0 0.0
6 1993 0 0.0
7 1992 0 0.0
8 1991 0 0.0
9 1990 0 0.0
10 1989 0 0.0
11 1988 0 0.0
12 1987 0 0.0
13 1986 0 0.0
14 1985 0 0.0
15 1984 0 0.0
16 1983 0 0.0
17 1982 0 0.0
18 1981 0 0.0
19 1980 0 0.0
20 1979 0 0.0
21 1979 0 0.0
22 1978 0 0.0
23 1977 0 0.0
24 1976 0 0.0
25 1976 0 0.0
26 1975 0 0.0
27 1974 0 0.0
28 1973 0 0.0
29 1972 0 0.0
30 1971 0 0.0
31 1970 0 0.0
32 1969 0 0.0
33 1969 0 0.0
34 1968 0 0.0
35 1968 0 0.0
36 1967 0 0.0
37 1966 0 0.0
38 1965 0 0.0
39 1964 0 0.0
40 1963 0 0.0
41 1962 0 0.0
42 1961 0 0.0
43 1960 0 0.0
44 1959 0 0.0
45 1958 0 0.0
46 1957 0 0.0
47 1957 0 0.0
48 1956 0 0.0
49 1955 0 0.0
50 1954 0 0.0
51 1953 0 0.0
52 1952 0 0.0
53 1951 0 0.0
54 1950 0 0.0
55 1949 0 0.0
56 1948 0 0.0
57 1947 0 0.0
58 1946 0 0.0
59 1945 0 0.0
60 1944 0 0.0
61 1943 0 0.0
62 1942 0 0.0
63 1941 0 0.0
64 1940 0 0.0
65 1939 0 0.0
66 1938 0 0.0
67 1937 0 0.0
68 1936 0 0.0
69 1935 0 0.0
70 1934 0 0.0
71 1933 0 0.0
72

400 1618 1 0.0006180469715698393
401 1617 1 0.0006184291898577613
402 1616 1 0.0006188118811881188
403 1615 1 0.0006191950464396285
404 1614 1 0.0006195786864931846
405 1613 1 0.0006199628022318661
406 1612 1 0.0006203473945409429
407 1611 1 0.0006207324643078833
408 1610 1 0.0006211180124223603
409 1609 1 0.0006215040397762585
410 1608 1 0.0006218905472636816
411 1607 1 0.0006222775357809583
412 1606 1 0.0006226650062266501
413 1605 1 0.0006230529595015577
414 1604 1 0.0006234413965087282
415 1604 1 0.0006234413965087282
416 1603 1 0.0006238303181534623
417 1603 1 0.0006238303181534623
418 1602 1 0.0006242197253433209
419 1601 1 0.0006246096189881324
420 1600 1 0.000625
421 1599 1 0.0006253908692933083
422 1598 1 0.0006257822277847309
423 1597 1 0.0006261740763932373
424 1596 1 0.0006265664160401002
425 1595 1 0.0006269592476489029
426 1594 1 0.0006273525721455458
427 1593 1 0.0006277463904582549
428 1592 1 0.000628140703517588
429 1591 1 0.0006285355122564425
430 1590 1 0.00062893081

650 1378 1 0.000725689404934688
651 1377 1 0.0007262164124909223
652 1376 1 0.0007267441860465116
653 1375 1 0.0007272727272727272
654 1374 1 0.000727802037845706
655 1373 1 0.0007283321194464676
656 1372 1 0.0007288629737609329
657 1371 1 0.0007293946024799417
658 1370 1 0.00072992700729927
659 1369 1 0.0007304601899196494
660 1368 1 0.0007309941520467836
661 1367 1 0.000731528895391368
662 1367 1 0.000731528895391368
663 1366 1 0.0007320644216691069
664 1365 1 0.0007326007326007326
665 1364 1 0.0007331378299120235
666 1364 1 0.0007331378299120235
667 1363 1 0.0007336757153338225
668 1362 1 0.0007342143906020558
669 1361 1 0.0007347538574577516
670 1360 1 0.0007352941176470588
671 1359 1 0.0007358351729212656
672 1358 1 0.0007363770250368188
673 1357 1 0.0007369196757553427
674 1356 1 0.0007374631268436578
675 1355 1 0.0007380073800738007
676 1354 1 0.0007385524372230429
677 1353 1 0.0007390983000739098
678 1352 1 0.0007396449704142012
679 1351 1 0.0007401924500370096
680 1350 1 0.000

902 1132 1 0.0008833922261484099
903 1131 1 0.0008841732979664014
904 1130 1 0.0008849557522123894
905 1129 1 0.0008857395925597874
906 1128 1 0.0008865248226950354
907 1128 1 0.0008865248226950354
908 1127 1 0.0008873114463176575
909 1126 1 0.0008880994671403197
910 1125 1 0.0008888888888888889
911 1124 1 0.0008896797153024911
912 1123 1 0.0008904719501335708
913 1122 1 0.00089126559714795
914 1121 1 0.0008920606601248885
915 1120 1 0.0008928571428571428
916 1119 1 0.0008936550491510277
917 1118 1 0.0008944543828264759
918 1117 1 0.0008952551477170994
919 1116 1 0.0008960573476702509
920 1115 1 0.0008968609865470852
921 1114 1 0.0008976660682226212
922 1113 1 0.0008984725965858042
923 1112 1 0.0008992805755395684
924 1111 1 0.0009000900090009
925 1111 1 0.0009000900090009
926 1110 1 0.0009009009009009009
927 1109 1 0.0009017132551848512
928 1108 1 0.0009025270758122744
929 1107 1 0.0009033423667570009
930 1106 1 0.0009041591320072332
931 1105 1 0.0009049773755656109
932 1104 1 0.00090

1154 897 1 0.0011148272017837235
1155 896 1 0.0011160714285714285
1156 895 1 0.0011173184357541898
1157 894 1 0.0011185682326621924
1158 894 1 0.0011185682326621924
1159 893 1 0.0011198208286674132
1160 892 1 0.0011210762331838565
1161 891 1 0.001122334455667789
1162 890 1 0.0011235955056179776
1163 889 1 0.0011248593925759281
1164 888 1 0.0011261261261261261
1165 887 1 0.0011273957158962795
1166 886 1 0.001128668171557562
1167 885 1 0.0011299435028248588
1168 884 1 0.0011312217194570137
1169 883 1 0.0011325028312570782
1170 882 1 0.0011337868480725624
1171 881 1 0.0011350737797956867
1172 880 1 0.0011363636363636363
1173 879 1 0.0011376564277588168
1174 878 1 0.0011389521640091116
1175 877 1 0.0011402508551881414
1176 876 1 0.001141552511415525
1177 875 1 0.001142857142857143
1178 874 1 0.0011441647597254005
1179 873 1 0.001145475372279496
1180 872 1 0.0011467889908256881
1181 871 1 0.001148105625717566
1182 870 1 0.0011494252873563218
1183 869 1 0.0011507479861910242
1184 868 1 0.001

...

841 1200 2 0.0016666666666666668
842 1199 2 0.0016680567139282735
843 1199 2 0.0016680567139282735
844 1198 2 0.001669449081803005
845 1197 2 0.001670843776106934
846 1196 2 0.0016722408026755853
847 1195 2 0.0016736401673640166
848 1195 2 0.0016736401673640166
849 1194 2 0.0016750418760469012
850 1193 2 0.0016764459346186086
851 1192 2 0.0016778523489932886
852 1191 2 0.0016792611251049538
853 1190 2 0.0016806722689075631
854 1189 2 0.001682085786375105
855 1188 2 0.0016835016835016834
856 1187 2 0.0016849199663016006
857 1186 2 0.0016863406408094434
858 1185 2 0.0016877637130801688
859 1184 2 0.0016891891891891893
860 1183 2 0.0016906170752324597
861 1182 2 0.001692047377326565
862 1181 2 0.001693480101608806
863 1180 2 0.001694915254237288
864 1179 2 0.0016963528413910093
865 1178 2 0.001697792869269949
866 1177 2 0.0016992353440951572
867 1177 2 0.0016992353440951572
868 1176 2 0.0017006802721088435
869 1175 2 0.001702127659574468
870 1174 2 0.0017035775127768314
871 1173 2 0.00170

1094 956 2 0.0020920502092050207
1095 955 2 0.0020942408376963353
1096 954 2 0.0020964360587002098
1097 953 2 0.002098635886673662
1098 952 2 0.0021008403361344537
1099 951 2 0.002103049421661409
1100 950 2 0.002105263157894737
1101 949 2 0.002107481559536354
1102 948 2 0.002109704641350211
1103 948 2 0.002109704641350211
1104 947 2 0.0021119324181626186
1105 946 2 0.0021141649048625794
1106 945 2 0.0021164021164021165
1107 945 2 0.0021164021164021165
1108 945 2 0.0021164021164021165
1109 944 2 0.00211864406779661
1110 943 2 0.0021208907741251328
1111 942 2 0.0021231422505307855
1112 941 2 0.0021253985122210413
1113 940 2 0.002127659574468085
1114 939 2 0.002129925452609159
1115 938 2 0.0021321961620469083
1116 937 2 0.0021344717182497333
1117 936 2 0.002136752136752137
1118 935 2 0.0021390374331550803
1119 934 2 0.0021413276231263384
1120 933 2 0.0021436227224008574
1121 932 2 0.002145922746781116
1122 932 2 0.002145922746781116
1123 931 2 0.0021482277121374865
1124 930 2 0.0021505376

1349 713 4 0.005610098176718092
1350 712 4 0.0056179775280898875
1351 711 4 0.005625879043600563
1352 710 4 0.005633802816901409
1353 710 4 0.005633802816901409
1354 709 4 0.005641748942172073
1355 708 4 0.005649717514124294
1356 707 4 0.005657708628005658
1357 706 4 0.0056657223796034
1358 705 4 0.005673758865248227
1359 704 4 0.005681818181818182
1360 703 4 0.005689900426742532
1361 702 4 0.005698005698005698
1362 701 4 0.005706134094151213
1363 700 4 0.005714285714285714
1364 699 4 0.005722460658082976
1365 698 4 0.0057306590257879654
1366 697 4 0.005738880918220947
1367 696 4 0.005747126436781609
1368 695 4 0.0057553956834532375
1369 694 4 0.005763688760806916
1370 693 4 0.005772005772005772
1371 692 4 0.005780346820809248
1372 691 4 0.005788712011577424
1373 690 4 0.005797101449275362
1374 689 4 0.005805515239477504
1375 688 4 0.005813953488372093
1376 687 4 0.005822416302765648
1377 686 4 0.0058309037900874635
1378 685 4 0.00583941605839416
1379 684 4 0.005847953216374269
1380 68

1610 462 5 0.010822510822510822
1611 461 5 0.010845986984815618
1612 461 5 0.010845986984815618
1613 460 5 0.010869565217391304
1614 460 5 0.010869565217391304
1615 459 5 0.010893246187363835
1616 458 5 0.010917030567685589
1617 457 5 0.010940919037199124
1618 456 5 0.010964912280701754
1619 455 5 0.01098901098901099
1620 454 5 0.011013215859030838
1621 453 5 0.011037527593818985
1622 452 5 0.011061946902654867
1623 451 5 0.011086474501108648
1624 450 5 0.011111111111111112
1625 449 5 0.011135857461024499
1626 448 5 0.011160714285714286
1627 447 5 0.011185682326621925
1628 446 5 0.011210762331838564
1629 445 5 0.011235955056179775
1630 444 5 0.01126126126126126
1631 443 5 0.011286681715575621
1632 442 5 0.011312217194570135
1633 441 5 0.011337868480725623
1634 440 5 0.011363636363636364
1635 439 5 0.011389521640091117
1636 438 5 0.01141552511415525
1637 437 5 0.011441647597254004
1638 436 5 0.011467889908256881
1639 435 5 0.011494252873563218
1640 434 5 0.01152073732718894
1641 433 5 0

1873 209 5 0.023923444976076555
1874 209 5 0.023923444976076555
1875 208 5 0.02403846153846154
1876 207 5 0.024154589371980676
1877 206 5 0.024271844660194174
1878 205 5 0.024390243902439025
1879 204 5 0.024509803921568627
1880 203 5 0.024630541871921183
1881 202 5 0.024752475247524754
1882 201 5 0.024875621890547265
1883 200 5 0.025
1884 199 5 0.02512562814070352
1885 198 5 0.025252525252525252
1886 197 5 0.025380710659898477
1887 196 5 0.025510204081632654
1888 195 5 0.02564102564102564
1889 194 5 0.02577319587628866
1890 193 5 0.025906735751295335
1891 192 5 0.026041666666666668
1892 191 5 0.02617801047120419
1893 190 5 0.02631578947368421
1894 189 5 0.026455026455026454
1895 188 5 0.026595744680851064
1896 187 5 0.026737967914438502
1897 186 5 0.026881720430107527
1898 186 5 0.026881720430107527
1899 185 5 0.02702702702702703
1900 184 5 0.02717391304347826
1901 183 5 0.0273224043715847
1902 182 5 0.027472527472527472
1903 181 5 0.027624309392265192
1904 180 5 0.027777777777777776
1